In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastbook import *
from fastai.imports import *
from fastai.tabular.all import *
import os

In [3]:
path = Path(os.path.expandvars("${HOME}/.fastai/data/titanic"))
path.ls()

(#4) [Path('/home/jmd/.fastai/data/titanic/gender_submission.csv'),Path('/home/jmd/.fastai/data/titanic/titanic.zip'),Path('/home/jmd/.fastai/data/titanic/train.csv'),Path('/home/jmd/.fastai/data/titanic/test.csv')]

In [4]:
df = pd.read_csv(f"{path}/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Explore Pandas

In [17]:
df.iloc[:,0] # Show all ids

0        1
1        2
2        3
3        4
4        5
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 891, dtype: int64

In [18]:
df.iloc[0,:]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

In [19]:
df.iloc[0]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

## Data Setup

In [25]:
def create_vars():
    cat_names  = ['Pclass', 'Name', 'Sex','Parch', 
                  'Ticket', 'Cabin', 'Embarked']
    cont_names = ['PassengerId', 'Age', 'Fare', 'SibSp']
    procs = [Categorify, FillMissing, Normalize]
    return cat_names, cont_names, procs

In [26]:
cat_names, cont_names, procs = create_vars()

In [27]:
dls = TabularDataLoaders.from_df(
    df, path, procs=procs, 
    cat_names=cat_names, cont_names=cont_names,
    y_names="Survived", y_block=CategoryBlock, bs=64)

In [28]:
dls.show_batch()

,Pclass,Name,Sex,Parch,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,SibSp,Survived
0,3,"Mannion, Miss. Margareth",female,0,36866,#na#,Q,True,727.999990,28.0,7.737500,3.755630e-09,1
1,3,"Henry, Miss. Delia",female,0,382649,#na#,Q,True,264.999998,28.0,7.750000,3.755630e-09,0
2,3,"Birkeland, Mr. Hans Martin Monsen",male,0,312992,#na#,S,False,408.999999,21.0,7.775000,3.755630e-09,0
3,1,"Kimball, Mr. Edwin Nelson Jr",male,0,11753,D19,S,False,622.000000,42.0,52.554200,1.000000e+00,1
4,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,0,350043,#na#,S,False,147.000014,27.0,7.795800,3.755630e-09,1
5,3,"Goodwin, Master. Harold Victor",male,2,CA 2144,#na#,S,False,481.000001,9.0,46.900002,5.000000e+00,0
6,3,"Lahoud, Mr. Sarkis",male,0,2624,#na#,C,True,522.999999,28.0,7.225000,3.755630e-09,0
7,3,"Rommetvedt, Mr. Knud Paust",male,0,312993,#na#,S,True,668.000005,28.0,7.775000,3.755630e-09,0
8,3,"Rice, Master. Eric",male,1,382652,#na#,Q,False,278.999994,7.0,29.125000,4.000000e+00,0
9,3,"Eklund, Mr. Hans Linus",male,0,347074,#na#,S,False,765.000004,16.0,7.775000,3.755630e-09,0


## Model Setup

In [29]:
learn = tabular_learner(dls, metris=)

In [30]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.652238,0.680840,00:00


In [31]:
learn.fit(5, lr=1e-3)

epoch,train_loss,valid_loss,time
0,0.308503,0.745237,00:00
1,0.189753,0.961839,00:00
2,0.119962,1.061945,00:00
3,0.080731,0.929796,00:00
4,0.056898,0.728306,00:00


## Data Setup with Tabular Pandas

In [12]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [13]:
cat_names, cont_names, procs = create_vars()
to = TabularPandas(df, procs=procs,
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Survived',
                   y_block = CategoryBlock,
                   splits=splits)

In [14]:
to.xs.iloc[:2]

,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare
243,3,501,2,1,1,659,0,3,1,-0.789902,-0.601913,-0.517923
476,2,677,2,2,1,253,0,3,1,0.112259,0.308244,-0.232166


In [15]:
dls = to.dataloaders(bs=64)

In [16]:
dls.show_batch()

,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,Survived
0,3,"Johnson, Miss. Eleanor Ileen",female,1,1,347742,#na#,S,False,173.000012,1.000000,11.133300,1
1,3,"Tikkanen, Mr. Juho",male,0,0,STON/O 2. 3101293,#na#,S,False,382.999997,32.000000,7.925000,0
2,3,"Lang, Mr. Fang",male,0,0,1601,#na#,S,False,509.999999,26.000000,56.495800,1
3,2,"Andrew, Mr. Edgardo Samuel",male,0,0,231945,#na#,S,False,145.000013,18.000000,11.500001,0
4,3,"Hirvonen, Miss. Hildur E",female,0,1,3101298,#na#,S,False,479.999999,2.000001,12.287500,1
5,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,1,0,PC 17569,B78,C,True,31.999991,29.000000,146.520793,1
6,3,"Youseff, Mr. Gerious",male,0,0,2628,#na#,C,False,203.999998,45.500000,7.225000,0
7,1,"Graham, Mrs. William Thompson (Edith Junkins)",female,0,1,PC 17582,C125,S,False,269.000002,57.999998,153.462493,1
8,3,"Coutts, Master. William Loch ""William""",male,1,1,C.A. 37671,#na#,S,False,349.000001,2.999999,15.900000,1
9,2,"Ponesell, Mr. Martin",male,0,0,250647,#na#,S,False,800.999997,34.000000,13.000000,0


In [17]:
learn = tabular_learner(dls, metrics=accuracy)

In [18]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.649969,0.673152,0.657303,00:00


In [19]:
learn.show_results()

,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,Survived,Survived_pred
0,2.0,822.0,1.0,1.0,1.0,141.0,0.0,3.0,1.0,-0.185880,-0.146835,-0.404135,1.0,0.0
1,3.0,182.0,2.0,1.0,1.0,655.0,0.0,3.0,1.0,-1.122888,0.004858,-0.498872,0.0,0.0
2,3.0,883.0,1.0,2.0,1.0,194.0,0.0,1.0,1.0,-1.300997,-1.170761,-0.366977,0.0,0.0
3,3.0,616.0,2.0,1.0,1.0,501.0,0.0,3.0,1.0,0.909878,-0.753605,-0.474673,0.0,0.0
4,3.0,29.0,2.0,2.0,6.0,334.0,0.0,3.0,1.0,-1.680447,0.687475,-0.020551,0.0,0.0
5,3.0,751.0,2.0,1.0,1.0,486.0,0.0,3.0,2.0,1.510027,-0.070988,-0.498872,0.0,0.0
6,3.0,877.0,1.0,2.0,1.0,189.0,0.0,1.0,1.0,1.482924,-1.132837,-0.366977,1.0,0.0
7,3.0,169.0,2.0,1.0,1.0,357.0,0.0,3.0,1.0,0.259392,-0.450220,-0.510286,0.0,0.0
8,3.0,656.0,1.0,1.0,1.0,282.0,0.0,2.0,2.0,0.902134,-0.070988,-0.497070,0.0,0.0


In [20]:
row, clas, probs = learn.predict(df.iloc[0])

In [21]:
row.show()

,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,Survived
0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,#na#,S,False,1.000006,22.0,7.249999,0


In [22]:
clas, probs

(tensor(0), tensor([0.5276, 0.4724]))

In [23]:
test_df = df.copy()
test_df.drop(['Survived'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [24]:
learn.get_preds(dl=dl)

(tensor([[0.5276, 0.4724],
         [0.5342, 0.4658],
         [0.5293, 0.4707],
         ...,
         [0.5302, 0.4698],
         [0.5285, 0.4715],
         [0.5326, 0.4674]]),
 None)